# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [63]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor

from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error

from sklearn.model_selection import GridSearchCV

In [64]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [65]:
spaceship.shape

(8693, 14)

In [66]:
spaceship = spaceship.dropna()

In [67]:
def cabin_category(cabin):
    return cabin.split('/')[0]

spaceship["Cabin"] = spaceship["Cabin"].apply(cabin_category)

In [68]:
spaceship = spaceship.drop(['PassengerId', 'Name'], axis=1)

In [69]:
spaceship_dummies = pd.get_dummies(spaceship, columns=['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP', 'Transported'], drop_first=True).astype(int)

spaceship_dummies

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True,Transported_True
0,39,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0
1,24,109,9,25,549,44,0,0,0,0,0,0,0,1,0,0,0,1,0,1
2,58,43,3576,0,6715,49,1,0,0,0,0,0,0,0,0,0,0,1,1,0
3,33,0,1283,371,3329,193,1,0,0,0,0,0,0,0,0,0,0,1,0,0
4,16,303,70,151,565,2,0,0,0,0,0,0,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,41,0,6819,0,1643,74,1,0,0,0,0,0,0,0,0,0,0,0,1,0
8689,18,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0
8690,26,0,0,1872,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1
8691,32,0,1049,0,353,3235,1,0,0,0,0,0,1,0,0,0,0,0,0,0


In [70]:
features = spaceship_dummies.drop("Transported_True", axis = 1)
target = spaceship_dummies['Transported_True']

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [71]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20)

normalizer = MinMaxScaler()

normalizer.fit(X_train)
X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

In [72]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)

display(X_train_norm)
display(X_test_norm)

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True
0,0.240506,0.019101,0.000000,0.073143,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.468354,0.001631,0.000000,0.008501,0.053686,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.341772,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.227848,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5279,0.506329,0.067552,0.000000,0.041756,0.000268,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
5280,0.227848,0.000116,0.000037,0.000000,0.010086,0.023997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
5281,0.632911,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5282,0.278481,0.000000,0.030771,0.035591,0.000000,0.000590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True
0,0.329114,0.000932,0.226737,0.000000,0.000134,0.037077,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.430380,0.028535,0.000000,0.050631,0.000000,0.000934,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.202532,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.227848,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.392405,0.000233,0.000739,0.000093,0.045787,0.000098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317,0.164557,0.001863,0.000000,0.008127,0.024277,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1318,0.417722,0.001048,0.050423,0.000000,0.248884,0.072925,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1319,0.518987,0.000000,0.015219,0.000000,0.179846,0.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1320,0.620253,0.001048,0.006612,0.000000,0.000134,0.386753,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [73]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier()
knn.fit(X_train_norm, y_train)

y_pred = knn.predict(X_test_norm)

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 0.7700453857791225


- Evaluate your model

In [74]:
knn.score(X_train_norm, y_train)

0.8217259651778955

In [75]:
knn.score(X_test_norm, y_test)

0.7700453857791225

**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [135]:
grid = {
    "n_neighbors": [25, 27, 29, 31, 33],      
    "weights": ["uniform", "distance"],     
    "p": [1, 2]                            
}

- Run Grid Search

In [136]:
knn = KNeighborsClassifier()

model = GridSearchCV(knn, param_grid=grid, scoring="accuracy", cv=10, verbose=2, n_jobs=-1)

In [137]:
model.fit(X_train_norm, y_train)

Fitting 10 folds for each of 20 candidates, totalling 200 fits
[CV] END ...............n_neighbors=25, p=1, weights=uniform; total time=   0.1s
[CV] END ...............n_neighbors=25, p=1, weights=uniform; total time=   0.1s
[CV] END ...............n_neighbors=25, p=1, weights=uniform; total time=   0.1s
[CV] END ...............n_neighbors=25, p=1, weights=uniform; total time=   0.1s
[CV] END ...............n_neighbors=25, p=1, weights=uniform; total time=   0.1s
[CV] END ...............n_neighbors=25, p=1, weights=uniform; total time=   0.1s
[CV] END ...............n_neighbors=25, p=1, weights=uniform; total time=   0.1s
[CV] END ...............n_neighbors=25, p=1, weights=uniform; total time=   0.1s
[CV] END ...............n_neighbors=25, p=1, weights=uniform; total time=   0.1s
[CV] END ..............n_neighbors=25, p=1, weights=distance; total time=   0.0s
[CV] END ..............n_neighbors=25, p=1, weights=distance; total time=   0.0s
[CV] END ..............n_neighbors=25, p=1, we

GridSearchCV(cv=10, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [25, 27, 29, 31, 33], 'p': [1, 2],
                         'weights': ['uniform', 'distance']},
             scoring='accuracy', verbose=2)

In [138]:
model.best_params_

{'n_neighbors': 29, 'p': 1, 'weights': 'uniform'}

In [139]:
best_model = model.best_estimator_

- Evaluate your model

In [150]:
print(f"Train Accuracy: {best_model.score(X_train_norm, y_train)}")

Train Accuracy: 0.788228614685844


In [151]:
print(f"Test Accuracy: {best_model.score(X_test_norm, y_test)}")

Test Accuracy: 0.7791225416036308


In [153]:
print(f"Overall Accuracy: {accuracy_score(y_test, y_pred)}")

Overall Accuracy: 0.7700453857791225
